<a href="https://colab.research.google.com/github/SBGnanapriya/rotary_screw/blob/main/compressor_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [76]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler

import joblib

In [77]:
df = pd.read_csv("compressor_data.csv")
df.head()

,Motor_Current_A,Oil_Temperature_C,Line_Pressure_bar,Filter_DeltaP_bar,Vibration_RMS_mm_s,Running_Hours,Status,Label,Reason
0,38.90,89.29,3.08,0.225,2.42,2156.0,Warning,1,Service interval warning (2000-3000 hrs)
1,59.33,94.59,2.90,0.237,4.81,1639.5,Warning,1,Multiple parameters in warning range
2,34.68,104.85,3.08,0.532,4.25,511.6,Critical,2,Filter choking (High temperature + High ΔP)
3,26.02,99.59,2.76,0.572,5.26,4040.1,Critical,2,Service critical (>3000 hrs) - Service required
4,27.87,66.94,2.82,0.047,1.59,542.7,Normal,0,NaN


In [78]:
print("\nDataset Shape:", df.shape)
print("\nColumn Names:", df.columns)
print("\nMissing Values:\n", df.isnull().sum())
print("\nLabel Distribution:\n", df['Label'].value_counts())


Dataset Shape: (11000, 9)

Column Names: Index(['Motor_Current_A', 'Oil_Temperature_C', 'Line_Pressure_bar',
       'Filter_DeltaP_bar', 'Vibration_RMS_mm_s', 'Running_Hours', 'Status',
       'Label', 'Reason'],
      dtype='object')

Missing Values:
 Motor_Current_A          0
Oil_Temperature_C        0
Line_Pressure_bar        0
Filter_DeltaP_bar        0
Vibration_RMS_mm_s       0
Running_Hours            0
Status                   0
Label                    0
Reason                3663
dtype: int64

Label Distribution:
 Label
1    5500
0    3663
2    1837
Name: count, dtype: int64


In [79]:
X = df[['Motor_Current_A', 'Oil_Temperature_C', 'Line_Pressure_bar', 'Vibration_RMS_mm_s']]
y = df['Label']

In [80]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

In [81]:
model = RandomForestClassifier(
    n_estimators=200,
    random_state=42,
    n_jobs=-1
)

model.fit(X_train, y_train)

RandomForestClassifier(n_estimators=200, n_jobs=-1, random_state=42)

In [82]:
y_pred = model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n")
print(classification_report(y_test, y_pred))

Accuracy: 0.8359090909090909

Classification Report:

              precision    recall  f1-score   support

           0       0.72      0.91      0.81       733
           1       0.89      0.77      0.82      1100
           2       0.99      0.89      0.94       367

    accuracy                           0.84      2200
   macro avg       0.87      0.86      0.86      2200
weighted avg       0.85      0.84      0.84      2200



In [83]:
joblib.dump(model, "compressor_fault_model.pkl")

['compressor_fault_model.pkl']

In [84]:
model = joblib.load("compressor_fault_model.pkl")

In [85]:
def diagnose_fault(current, temperature, pressure, vibration):
    if current > 60 and pressure < 2.8:
        return "Air leak or Airend wear"
    elif temperature > 90 and pressure < 2.8:
        return "Filter choking"
    elif current > 60 and vibration > 4:
        return "Bearing or alignment issue"
    else:
        return "No specific fault detected"

In [86]:
def safe_float(prompt):
    while True:
        try:
            return float(input(prompt))
        except ValueError:
            print("Invalid input. Please enter a number.")

In [87]:
def get_user_input():
    current = safe_float("Enter Motor Current (A): ")
    temperature = safe_float("Enter Oil Temperature (°C): ")
    pressure = safe_float("Enter Line Pressure (bar): ")
    vibration = safe_float("Enter Vibration (mm/s): ")

    return current, temperature, pressure, vibration

In [89]:
status_map = {
    0: "NORMAL",
    1: "WARNING",
    2: "CRITICAL"
}

def predict_fault(model):
    current, temperature, pressure, vibration = get_user_input()

    user_data = pd.DataFrame([[current, temperature, pressure, vibration]],columns=['Motor_Current_A', 'Oil_Temperature_C', 'Line_Pressure_bar', 'Vibration_RMS_mm_s'])

    prediction = model.predict(user_data)[0]
    status = status_map[prediction]

    fault = diagnose_fault(current, temperature, pressure, vibration)

    print("\n==============================")
    print(" MACHINE DIAGNOSIS RESULT")
    print("==============================")
    print("Machine Status :", status)
    print("Likely Cause   :", fault)
    print("==============================\n")

In [90]:
predict_fault(model)

Enter Motor Current (A): 55
Enter Oil Temperature (°C): 78
Enter Line Pressure (bar): 3
Enter Vibration (mm/s): 3

 MACHINE DIAGNOSIS RESULT
Machine Status : NORMAL
Likely Cause   : No specific fault detected

